In [2]:
import pandas as pd 

In [3]:
df=pd.read_csv("kidney_disease.csv")
df.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [4]:
df.shape

(400, 26)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              400 non-null    int64  
 1   age             391 non-null    float64
 2   bp              388 non-null    float64
 3   sg              353 non-null    float64
 4   al              354 non-null    float64
 5   su              351 non-null    float64
 6   rbc             248 non-null    object 
 7   pc              335 non-null    object 
 8   pcc             396 non-null    object 
 9   ba              396 non-null    object 
 10  bgr             356 non-null    float64
 11  bu              381 non-null    float64
 12  sc              383 non-null    float64
 13  sod             313 non-null    float64
 14  pot             312 non-null    float64
 15  hemo            348 non-null    float64
 16  pcv             330 non-null    object 
 17  wc              295 non-null    obj

In [6]:
df.isnull().sum()

id                  0
age                 9
bp                 12
sg                 47
al                 46
su                 49
rbc               152
pc                 65
pcc                 4
ba                  4
bgr                44
bu                 19
sc                 17
sod                87
pot                88
hemo               52
pcv                70
wc                105
rc                130
htn                 2
dm                  2
cad                 2
appet               1
pe                  1
ane                 1
classification      0
dtype: int64

In [7]:
df.select_dtypes(include=['object']).columns

Index(['rbc', 'pc', 'pcc', 'ba', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [8]:
df["classification"].value_counts()

classification
ckd       248
notckd    150
ckd\t       2
Name: count, dtype: int64

In [9]:
df["al"].value_counts()

al
0.0    199
1.0     44
3.0     43
2.0     43
4.0     24
5.0      1
Name: count, dtype: int64

In [10]:
df.drop(columns=['id'], inplace=True)
for col in ['pcv', 'wc', 'rc']:
    df[col] = pd.to_numeric(df[col], errors='coerce')  
for col in df.select_dtypes(include=['float64']).columns:
    df[col] = df[col].fillna(df[col].median())  
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].fillna(df[col].mode()[0])  
df.isnull().sum().sum()  

np.int64(0)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

categorical_cols = df.select_dtypes(include=['object']).columns
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  

X = df.drop(columns=['classification'])  
y = df['classification']  

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((320, 24), (80, 24), (320,), (80,))

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

accuracy,classification_rep


(0.975,
 '              precision    recall  f1-score   support\n\n           0       1.00      0.96      0.98        50\n           2       0.94      1.00      0.97        30\n\n    accuracy                           0.97        80\n   macro avg       0.97      0.98      0.97        80\nweighted avg       0.98      0.97      0.98        80\n')

In [13]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[48,  2],
       [ 0, 30]])

In [14]:
label_encoders

{'rbc': LabelEncoder(),
 'pc': LabelEncoder(),
 'pcc': LabelEncoder(),
 'ba': LabelEncoder(),
 'htn': LabelEncoder(),
 'dm': LabelEncoder(),
 'cad': LabelEncoder(),
 'appet': LabelEncoder(),
 'pe': LabelEncoder(),
 'ane': LabelEncoder(),
 'classification': LabelEncoder()}

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score

# Encode Categorical Variables
categorical_cols = df.select_dtypes(include=['object']).columns
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  

# Define Features and Target
X = df.drop(columns=['classification'])  
y = df['classification']  

# Scale Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply Forward Feature Selection
base_model = RandomForestClassifier(n_estimators=100, random_state=42)
selector = SequentialFeatureSelector(base_model, direction='forward', n_features_to_select=5, n_jobs=-1,cv=2)  # Select top 3
selector.fit(X_scaled, y)

# Get Selected Features
selected_features = X.columns[selector.get_support()]
print("✅ Selected Features using Forward Selection:", list(selected_features))

# Use Only Selected Features for Training
X_selected = selector.transform(X_scaled)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42, stratify=y)

print("\n🚀 Final Shapes After Feature Selection:")
print("X_train:", X_train.shape, "X_test:", X_test.shape)
print("y_train:", y_train.shape, "y_test:", y_test.shape)

# Train the Random Forest Classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Make Predictions
y_pred = classifier.predict(X_test)

# Compute MSE and RMSE
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5  # Square root of MSE

# Compute Accuracy
accuracy = accuracy_score(y_test, y_pred)

print("\n✅ Model Evaluation:")
print("📉 Mean Squared Error (MSE):", mse)
print("📊 Root Mean Squared Error (RMSE):", rmse)
print("🎯 Accuracy:", accuracy)


✅ Selected Features using Forward Selection: ['sg', 'al', 'sc', 'hemo', 'htn']

🚀 Final Shapes After Feature Selection:
X_train: (320, 5) X_test: (80, 5)
y_train: (320,) y_test: (80,)

✅ Model Evaluation:
📉 Mean Squared Error (MSE): 0.05
📊 Root Mean Squared Error (RMSE): 0.22360679774997896
🎯 Accuracy: 0.9875
